# **Projeto de Probabilidade e inferência**: Análise de dados de performance dos times no cenário competitivo de Valorant

Autores:

$\to$ Caio Vinícius de Azevedo   - 20220040815;

$\to$ Davi Matias Soares Genuino - 20220042186;

$\to$ Hannah Beatryz Lima Santos - 20220034925;

# **1 - Descrição do Projeto:**

O presente projeto consiste na análise do desempenho de alguns times profissionais no jogo Valorant, desenvolvido pela Riot Games.
Valorant é um jogo de tiro, com visão em primeira pessoa, que envolve personagens com habilidades especiais. Para via de explicação dos dados aqui descritos, apresentaremos uma breve descrição das mecânicas de vitória e derrota do jogo.

De modo geral, os jogos competitivios envolvem diversos mapas. A partida em um desses mapas é dividida em rounds, onde os dois times são postos nas posições de ataque e defesa. A cada 12 rounds, essas posições são invertidas para que o time atacante passe a ser o defensor e vice-versa. Vence a partida (e o mapa) o time que atingir 13 rounds vitoriosos primeiro.

No primeiro round da partida e no primeiro round após a inversão, nenhum dos times tem acesso a armas potentes ou recursos em abundância. Hipoteticamente, o resultado desse round, no entanto, poderá garantir essas ferramentas para o time que vencer. Por essa razão, os chamados *Rounds de pistola* ou simplesmente *pistol*'s possuem destaque dentre os outros. Sempre haverão dois rounds pistol por partida, independentemente de qual equipe começou defendendo ou atacando.

Os arquivos com os dados estão disponíveis [no site kaggle](https://www.kaggle.com/datasets/anud3ep/valorant), e as informações neles dispostas foram extraídas do site [The Spike](https://www.thespike.gg) em 14 de abril de 2022, relativas aos três meses anteriores à referida data.


## Hipóteses

Antes da análise, formulamos as seguintes hipóteses sobre os dados:

1. Times que jogaram em mais mapas acabarão por ter uma taxa de vitória maior em todos os aspectos.
  - A hipótese se fundamenta no fato de que, em cenários competitivos, times que não jogarem em muitos mapas provavelmente foram eliminados mais cedo que os outros.

2. Times que tem alta taxa de vitória enquanto estão na função de atacantes possuirão taxa de vitória significativamente menor quando estiverem defendendo e vice-versa.
  - Hipótese baseada na ideia de que times podem possuir estratégias com qualidades discrepantes para cada situação. Isso pode estar relacionado com características individuais dos seus componentes, ou do modo segundo o qual realizam seus treinos
3. Taxas de vitórias altas em rounds de pistola aumentam a chance de vencer o mapa.
  - Hipótese baseada nas características dos rounds de pistola em relação aos demais.

Com essas hipóteses em mente, partiremos para a análise dos dados.

# **2 - Pré-processamento e Análise preliminar:**



Utilizaremos a linguagem `Python`, aliada às bibliotecas `pandas`, `matplotlib`, `seaborn`, `statsmodels` e `numpy` como forma principal de manipulação dos dados. O comando abaixo serve para realizar a configuração inicial do projeto, em conjunto com a importação das bibliotecas e da nossa base de dados.

In [207]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style("dark")


df=pd.read_csv('sample_data/teams.csv', encoding= 'unicode_escape')

O bloco de códigos abaixo realiza a conversão das entrada da base para valores do tipo `float`. Em seguida, são exibidas 10 entradas da base de dados, representando o desempenho de 10 times até a data de coleta dos dados. Para cada tipo de round são calculadas também as taxas de vitória (ou *winrate*s, como também as chamaremos), tomadas com base no cálculo: $$winrate = \frac{vitorias}{round}$$

In [ ]:
# Converting number of matches played/won from int --> float
df['maps_won']=pd.to_numeric(df['maps_won'],downcast='float')
df['atk_won']=pd.to_numeric(df['atk_won'],downcast='float')
df['def_won']=pd.to_numeric(df['def_won'],downcast='float')
df['pistol_won']=pd.to_numeric(df['pistol_won'],downcast='float')

df['maps_played']=pd.to_numeric(df['maps_played'],downcast='float')
df['atk_played']=pd.to_numeric(df['atk_played'],downcast='float')
df['def_played']=pd.to_numeric(df['def_played'],downcast='float')
df['pistol_played']=pd.to_numeric(df['pistol_played'],downcast='float')

# Converting winrate from string (percentage) --> float
df['maps_won%']=pd.to_numeric(df['maps_won']/df['maps_played'],downcast='float')
df['atk_won%']=pd.to_numeric(df['atk_won']/df['atk_played'],downcast='float')
df['def_won%']=pd.to_numeric(df['def_won']/df['def_played'],downcast='float')
df['pistol_won%']=pd.to_numeric(df['pistol_won']/df['pistol_played'],downcast='float')
df.head(22).T

A seguir, podemos verificar mais algumas informações sobre nossos dados. Ao total, temos à nossa disposição um total de 309 entradas, cada uma com o desempenho de um time profissional de Valorant no ano de 2022 até a data de coleta.

In [ ]:
df.info()

Agora, possuimos todos os dados em sua configuração correta para a realização da análise. Começaremos pela verificação simples de algumas variáveis, como as Médias e Medianas, Desvios-Padrões e Variâncias das taxas de vitória (winrate) em cada tipo de round.

In [ ]:
# Getting the Means:
print("A winrate média em rounds de pistola é 	:	{:.3f}%".format(100*np.mean(df['pistol_won%'])))
print("A winrate média em rounds de ataque é 	:	{:.3f}%".format(100*np.mean(df['atk_won%'])))
print("A winrate média em rounds de defesa é 	:	{:.3f}%".format(100*np.mean(df['def_won%'])))
print("A winrate média em mapas é 		:	{:.3f}%".format(100*np.mean(df['maps_won%'])))

In [ ]:

# Getting the Medians:
print("A winrate mediana em rounds de pistola é   :  {:.3f}%".format(100*np.median(df['pistol_won%'])))
print("A winrate mediana em rounds de ataque é    :  {:.3f}%".format(100*np.median(df['atk_won%'])))
print("A winrate mediana em rounds de defesa é    :  {:.3f}%".format(100*np.median(df['def_won%'])))
print("A winrate mediana é                        :  {:.3f}%".format(100*np.median(df['maps_won%'])))

In [ ]:
# Getting the Standard Deviations:
print("O desvio padrão da winrate em rounds de pistola é   :  {:.3f}%".format(100*np.std(df['pistol_won%'])))
print("O desvio padrão da winrate em rounds de ataque é    :  {:.3f}%".format(100*np.std(df['atk_won%'])))
print("O desvio padrão da winrate em rounds de defesa é    :  {:.3f}%".format(100*np.std(df['def_won%'])))
print("O desvio padrão da winrate é                        :  {:.3f}%".format(100*np.std(df['maps_won%'])))

In [ ]:
# Getting the Variances:
print("A variância da winrate em rounds de pistola é   :  {:.3f}%".format(100*np.var(df['pistol_won%'])))
print("A variância da winrate em rounds de ataque é    :  {:.3f}%".format(100*np.var(df['atk_won%'])))
print("A variância da winrate em rounds de defesa é    :  {:.3f}%".format(100*np.var(df['def_won%'])))
print("A variância da winrate é                        :  {:.3f}%".format(100*np.var(df['maps_won%'])))

A princípio, é visível que as taxas indicam uma taxa de vitória ligeiramente maior em rounds de defesa do que de ataque. Notamos também que em todas as ocasiões, a média das winrates dos times profissionais está consideravelmente acima de 50%. Podemos ver como se dá a distribuição desses dados por meio de gráficos-violino

In [ ]:
plt.figure(figsize=(7,7))
plt.tight_layout()

p = plt.subplot(1, 2, 1)
p.set_xlim(left=0, right=5)
plt.title('Rounds de pistola jogados')
sns.violinplot(data=df['pistol_played'],color='white')

plt.subplot(1, 2, 2)
plt.title('Rounds de Pistola vencidos')
sns.violinplot(data=df['pistol_won'],color='red')

plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.tight_layout()

plt.subplot(1, 2, 1)
plt.title('Rounds de ataque jogados')
sns.violinplot(data=df['atk_played'],color='white')

plt.subplot(1, 2, 2)
plt.title('Rounds de ataque vencidos')
sns.violinplot(data=df['atk_won'],color='green')

plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.tight_layout()

plt.subplot(1, 2, 1)
plt.title('Rounds de defesa jogados')
sns.violinplot(data=df['def_played'],color='white')

plt.subplot(1, 2, 2)
plt.title('Rounds de defesa vencidos')
sns.violinplot(data=df['def_won'],color='blue')

plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.tight_layout()

plt.subplot(1, 2, 1)
plt.title('Mapas jogados')
sns.violinplot(data=df['maps_played'],color='white')

plt.subplot(1, 2, 2)
plt.title('Mapas vencidos')
sns.violinplot(data=df['maps_won'],color='yellow')

plt.show()

A seguir, veremos como se dá a dispersão do número de vitórias e da taxa de vitórias em relação ao número de rounds jogados, por tipo de round:

> Rounds de pistola:

In [ ]:
plt.figure(figsize=(14,7))
plt.subplot(1, 2, 1)
plt.title('Rounds de pistola: vencidos/jogados')
sns.scatterplot(data=df, x='pistol_played', y='pistol_won',)
plt.subplot(1, 2, 2)
plt.title('Rounds de pistola: winrate/jogados')
sns.scatterplot(data=df, x='pistol_played', y='pistol_won%')
plt.show()

> Rounds de ataque:

In [ ]:
plt.figure(figsize=(14,7))
plt.subplot(1, 2, 1)
plt.title('Rounds de ataque: vencidos/jogados')
sns.scatterplot(data=df, x='atk_played', y='atk_won',)
plt.subplot(1, 2, 2)
plt.title('Rounds de ataque: winrate/jogados')
sns.scatterplot(data=df, x='atk_played', y='atk_won%')
plt.show()

> Rounds de defesa:

In [ ]:
plt.figure(figsize=(14,7))
plt.subplot(1, 2, 1)
plt.title('Rounds de defesa: vencidos/jogados')
sns.scatterplot(data=df, x='def_played', y='def_won',)
plt.subplot(1, 2, 2)
plt.title('Rounds de defesa: winrate/jogados')
sns.scatterplot(data=df, x='def_played', y='def_won%')
plt.show()

> Mapas:

In [ ]:
plt.figure(figsize=(14,7))
plt.subplot(1, 2, 1)
plt.title('Mapas: vencidos/jogados')
sns.scatterplot(data=df, x='maps_played', y='maps_won',)
plt.subplot(1, 2, 2)
plt.title('Mapas: winrate/jogados')
sns.scatterplot(data=df, x='maps_played', y='maps_won%')
plt.show()

Em seguida, observaremos como esses atributos se distribuem em relação aos países dos quais cada time se apresenta:



> Rounds de pistola:

In [ ]:
plt.figure(figsize=(30,7))
plt.subplot()
plt.title('Rounds de Pistola jogados por país')
sns.barplot(x='country',y='pistol_played',data=df,linewidth=5)
plt.show()
plt.figure(figsize=(30,7))
plt.subplot()
plt.title('Taxa de vitória em rounds de pistola por país')
sns.barplot(x='country',y='pistol_won%',data=df,linewidth=5)
plt.show()

> Rounds de ataque:

In [ ]:
plt.figure(figsize=(30,7))
plt.subplot()
plt.title('Rounds de ataque jogados por país')
sns.barplot(x='country',y='atk_played',data=df,linewidth=5)
plt.show()
plt.figure(figsize=(30,7))
plt.subplot()
plt.title('Taxa de vitória em rounds de ataque por país')
sns.barplot(x='country',y='atk_won%',data=df,linewidth=5)
plt.show()

> Rounds de defesa:

In [ ]:
plt.figure(figsize=(30,7))
plt.subplot()
plt.title('Rounds de defesa jogados por país')
sns.barplot(x='country',y='def_played',data=df,linewidth=5)
plt.show()
plt.figure(figsize=(30,7))
plt.subplot()
plt.title('Taxa de defesa em rounds de ataque por país')
sns.barplot(x='country',y='def_won%',data=df,linewidth=5)
plt.show()

> Mapas:

In [ ]:
plt.figure(figsize=(30,7))
plt.subplot()
plt.title('Mapas jogados por país')
sns.barplot(x='country',y='maps_played',data=df,linewidth=5)
plt.show()
plt.figure(figsize=(30,7))
plt.subplot()
plt.title('Taxa de vitória em mapas por país')
sns.barplot(x='country',y='maps_won%',data=df,linewidth=5)
plt.show()

Para finalizar a nossa descrição dos dados, apresentamos aqui uma tabela de correlação entre todas as variáveis, demonstrando sua taxa de dependência.

In [ ]:
corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(8, 8))
    ax = sns.heatmap(corr,mask=mask,square=True,linewidths=.8,cmap="winter",annot=True)

# **3 - Amostragem**

Como a nossa base de dados contém apenas 309 entradas, não necessitamos realizar um processo de amostragem para obter uma amostra que seja mais fácil de trabalhar. No entanto, faremos esse processo ainda assim, para fins de exercício.

Utilizaremos um processo de amostragem probabilistico e estratificado por região. Escolheremos uma amostra com tamanho equivalente a 30% do nosso espaço amostral e realizaremos cálculos a respeito dele.

In [184]:
# Coletando os países disponíveis
unique_countries = df['country'].unique()
# Tamanho da amostra
sample_size = int(len(df) * 0.3)
## criando a amostra estratificada
sts = pd.DataFrame()
# Para cada país único
for country in unique_countries:
    estrato = df[df['country'] == country]            # Colete todos as linhas com aquele país
    sample = df.sample(n=sample_size, replace=False)  # Faça uma amostra simples dessas linhas
    sts = strat_sample.append(sample)        # Inclua na amostra estratificada

A partir dessa amostra, podemos repetir os processos realizados na sessão 2, para encontrar estimativas das média, mediana, desvio padrão e variância de cada uma das taxas de vitória. O bloco de código abaixo realiza essas tarefas e expõe a comparação das estimativas com os valores reais obtidos na sessão 2.

In [ ]:
# Comparação
print("\nMédias")
print("			   Base		   Amostra")
print("Rounds de pistola	: {:.3f}%	: {:.3f}%".format(100*np.mean(df['pistol_won%']), 100*np.mean(sts['pistol_won%'])))
print("Rounds de ataque	: {:.3f}%	: {:.3f}%".format(100*np.mean(df['atk_won%']), 100*np.mean(sts['atk_won%'])))
print("Rounds de defesa	: {:.3f}%	: {:.3f}%".format(100*np.mean(df['def_won%']), 100*np.mean(sts['def_won%'])))
print("Mapas			: {:.3f}%	: {:.3f}%".format(100*np.mean(df['maps_won%']), 100*np.mean(sts['maps_won%'])))

print("\nMedianas")
print("			Base		   Amostra")
print("Rounds de pistola	: {:.3f}%	: {:.3f}%".format(100*np.median(df['pistol_won%']), 100*np.median(sts['pistol_won%'])))
print("Rounds de ataque	: {:.3f}%	: {:.3f}%".format(100*np.median(df['atk_won%']), 100*np.median(sts['atk_won%'])))
print("Rounds de defesa	: {:.3f}%	: {:.3f}%".format(100*np.median(df['def_won%']), 100*np.median(sts['def_won%'])))
print("Mapas			: {:.3f}%	: {:.3f}%".format(100*np.median(df['maps_won%']), 100*np.median(sts['maps_won%'])))

print("\nDesvios Padrões")
print("			Base		   Amostra")
print("Rounds de pistola	: {:.3f}%	: {:.3f}%".format(100*np.std(df['pistol_won%']), 100*np.std(sts['pistol_won%'])))
print("Rounds de ataque	: {:.3f}%	: {:.3f}%".format(100*np.std(df['atk_won%']), 100*np.std(sts['atk_won%'])))
print("Rounds de defesa	: {:.3f}%	: {:.3f}%".format(100*np.std(df['def_won%']), 100*np.std(sts['def_won%'])))
print("Mapas			: {:.3f}%	: {:.3f}%".format(100*np.std(df['maps_won%']), 100*np.std(sts['maps_won%'])))

print("\nVariância")
print("			Base		   Amostra")
print("Rounds de pistola	: {:.3f}%	: {:.3f}%".format(100*np.var(df['pistol_won%']), 100*np.var(sts['pistol_won%'])))
print("Rounds de ataque	: {:.3f}%	: {:.3f}%".format(100*np.var(df['atk_won%']), 100*np.var(sts['atk_won%'])))
print("Rounds de defesa	: {:.3f}%	: {:.3f}%".format(100*np.var(df['def_won%']), 100*np.var(sts['def_won%'])))
print("Mapas			: {:.3f}%	: {:.3f}%".format(100*np.var(df['maps_won%']), 100*np.var(sts['maps_won%'])))

Como pudemos ver, os dados da nossa amostra possuem valures bastante similares com os valores reais da base, om diferença sempre inferior a 5% em todos os casos.

Podemos criar ainda uma tabela de correlação, similar à feita na análise preliminar, para verificar a dependência entre os dados:

In [ ]:
corrS = sts.corr()
mask = np.zeros_like(corrS)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(8, 8))
    ax = sns.heatmap(corrS,mask=mask,square=True,linewidths=.8,cmap="summer",annot=True)

# **4 -Teste de Hipóteses**

Com base nos dados da nossa base, verificaremos as hipóteses que fizemos no inicio deste roteiro.

## **Hipótese 1:** Times que jogaram em mais mapas acabarão por ter uma taxa de vitória maior em todos os aspectos.

Observando na tabela de correlação, na primeira coluna (que relaciona com o número de mapas jogados por equipe) vemos que a relação entre a taxa de vitória nos mapas (`maps_won%`), nos rounds de ataque (`atk_won%`), defesa (`def_won%`) e pistola (`pistol_won%`) são valores sempre inferiores a 18%. Dentre elas, o nível de correlação da winrate para rounds de pistola é o menor.

Ainda assim, todos os valores de correlação são superiores a 5%. Portanto, para verificar se a hipótese está correta, utilizaremos um modelo de regressão linear, utilizando o test t de student:

In [189]:
## Criando uma matriz nova com base nas antigas
modelArray = [df['maps_played'],df['maps_won%'],df['atk_won%'], df['def_won%'], df['pistol_won%']]
modelArray[0] = sm.add_constant(modelArray[0])

## Criando o modelo de regressão para:

In [ ]:
# Mapas:
model = sm.OLS(modelArray[1], modelArray[0])
results = model.fit()
print(results.summary())

A princípio, temos que o p-valor deste caso (taxa de vitória de mapas com base no número de mapas jogados) para a situação constante (`const`) é igual a 0 e, portanto, podemos aceitar o modelo.

No entanto, vemos que a taxa de vitória (`maps_won%`) possui um p-valor inferior a 0.05. Dito isso, não aceitaremos a hipótese de que a quantidade de mapas jogados afeta a taxa de vitória em mapas.

In [ ]:
# Rounds de Ataque
model = sm.OLS(modelArray[2], modelArray[0])
results = model.fit()
print(results.summary())

Assim como no caso anterior, também podemos aceitar o modelo - já que o p-valor da hipótese nula foi zero - porém, em razão do p-valor da variável ligada ter sido menor que 5%, constatamos que o impacto dessa variável é negligível.

In [ ]:
# Rounds de Defesa
model = sm.OLS(modelArray[3], modelArray[0])
results = model.fit()
print(results.summary())

Pelos mesmos motivos, também podemos aceitar esse modelo, porém, como o p-valor do teste é inferior a 5%, a relação entre as variáveis é desprezível.

In [ ]:
# Rounds de Pistola
model = sm.OLS(modelArray[4], modelArray[0])
results = model.fit()
print(results.summary())

Nesse caso, nós realmente podemos aceitar nosso modelo de regressão, uma vez que o p-valor do caso constante é 0. No entanto, diferentemente do primeiro caso, para a winrate em rounds de pistola o p-valor possui um tamanho significativo (16.3%).

Portanto, vemos que o número de mapas em que certo time jogou realmente impacta a taxa de vitória desse time em rounds de pistola, mesmo que o impacto seja pequeno, visto que o coeficiente obtido é bastante próximo de 0.

A hipótese de que o número de mapas afeta todas as taxas de vitória está incorreta, no entanto. Ela afeta apenas a taxa de vitória em rounds de pistola.

Uma explicação para isso no contexto do jogo pode ser o fato de que os rounds de pistola são consideravelmente menos complexos que os outros rounds, em razão da noção de que todos os competidores tem acesso a um número limitado de recursos. Além disso, em todos os mapas serão jogados exatamente dois rounds de pistola, de modo que as situações encontradas nesses cenários podem ser mais facilmente entendidas por times com mais partidas.

## **Hipótese 2:** Times que tem alta taxa de vitória enquanto estão na função de atacantes possuirão taxa de vitória significativamente menor quando estiverem defendendo e vice-versa.

Assim como na Hipótese anterior, realizaremos uma regressão linear nas variáveis para verificar a sua dependência.


In [195]:
## Criando uma matriz nova com base nas antigas PARA CADA CASO
model_atk_def = [df['atk_won%'],df['def_won%']]
model_def_atk = [df['def_won%'],df['atk_won%']]

## Adicionando um valor constante em cada caso
model_atk_def[0] = sm.add_constant(model_atk_def[0])
model_def_atk[0] = sm.add_constant(model_def_atk[0])

Após adicionados os valores constantes, vejamos os resultados obtidos nos modelos:

In [ ]:
model = sm.OLS(model_atk_def[1], model_atk_def[0])
results = model.fit()
print(results.summary())

In [ ]:
model = sm.OLS(model_def_atk[1], model_def_atk[0])
results = model.fit()
print(results.summary())

Note que, em ambos os casos, pudemos aceitar o modelo de regressão linear em decorrência do p-valor do caso constante ser igual a 0. No entanto, as relações entre as taxas de vitória também resultaram em 0, confirmando que elas são completamente diferentes.

Assim, a hipótese 2 também se mostrou equivocada.

## **Hipótese 3:** Taxas de vitórias altas em rounds de pistola aumentam a chance de vencer o mapa.

Da mesma maneira que nas hipóteses anteriores, também verificaremos esta hipótese utilizando regressão linear.

In [205]:
## Criando o modelo e adicionando um caso constante:
model_map_pistol = [df['pistol_won%'], df['maps_won%']]
model_map_pistol[0] = sm.add_constant(model_map_pistol[0])

In [ ]:
## Testando o modelo
model = sm.OLS(model_map_pistol[1], model_map_pistol[0])
results = model.fit()
print(results.summary())

Assim sendo, notamos que o valor do caso constante possui p-valor maior que 5%. Nesse caso, concluimos que não temos evidência suficiente para rejeitar a hipótese nula.